In [16]:
import requests

APIkey = {"X-Api-Key": }

In [75]:
import pandas as pd
url = 'https://newsapi.org/v2/everything'

params = {
    'q': 'Donald John Trump and bitcoin', #記事に含む単語
    'sortBy': 'publishedAt', #最新の記事を持ってくる
    'pageSize': 1 #読み込む記事の数
}

response = requests.get(url, headers=APIkey , params=params)

if response.ok:
    data = response.json()
    df = pd.DataFrame(data['articles'])

news = df[['description']]
news

,description
0,The latest edition of Wharton’s Consumer Cryptocurrency Confidence Report finds increasing owner...


In [87]:
from openai import OpenAI
client = OpenAI(api_key = )

In [88]:
def judge(summary):
    prompt = f"""
    あなたは金融アナリストです。この記事の要約文をもとに、仮想通貨市場に与える影響を判断してください。
    要約: "{summary}"
    出力は 'buy', 'sell', 'hold' のいずれかで答えてください。
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip().lower()
    except Exception as e:
        return "error"   

judgement = news.apply(judge)
print(judgement)

description    buy
dtype: object


In [82]:
import json
import requests
import time
import hmac
import hashlib

access_key = 
secret_key = 

class Coincheck:
    def __init__(self, access_key, secret_key, url='https://coincheck.com'):
        self.access_key = access_key
        self.secret_key = secret_key
        self.url = url

    def get(self, path, params=None):
        if params is not None:
            params = json.dumps(params)
        else:
            params = ''
        nonce = str(int(time.time()))
        message = nonce + self.url + path + params
    
        signature = self.getSignature(message)
    
        response = requests.get(
            self.url+path,
            headers=self.getHeader(self.access_key, nonce, signature)
        )
        
        try:
            return response.json()  # JSONに変換を試みる
        except Exception as e:
            print('JSONじゃないレスポンスが返った:', response.text)
            raise e

    def post(self, path, params):
        params = json.dumps(params)
        nonce = str(int(time.time()))
        message = nonce + self.url + path + params
        
        signature = self.getSignature(message)

        return requests.post(
            self.url+path,
            data=params,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()

    def delete(self, path):
        nonce = str(int(time.time()))
        message = nonce + self.url + path

        signature = self.getSignature(message)

        return requests.delete(
            self.url+path,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()
    
    def getSignature(self, message):
        signature = hmac.new(
            bytes(self.secret_key.encode('ascii')),
            bytes(message.encode('ascii')),
            hashlib.sha256
        ).hexdigest()
        
        return signature
    
    def getHeader(self, access_key, nonce, signature):
        headers = {
            'ACCESS-KEY': access_key,
            'ACCESS-NONCE': nonce,
            'ACCESS-SIGNATURE': signature,
            'Content-Type': 'application/json' 
        }
        
        return headers
        

client = Coincheck(access_key, secret_key)

In [90]:
if (judgement == "sell").any:
    print("売るよ")
    # params = {
    # 'amount': 0.001,
    # 'order_type': 'market_sell',
    # 'pair': 'btc_jpy'
    # }
elif (judgement == "buy").any:
    print("購入するよ")
    # params = {
    # 'amount': 0.001,
    # 'order_type': 'market_sell',
    # 'pair': 'btc_jpy'
    # }
else:
    print("保持！")
    
# response = client.post("/api/exchange/orders", params)

print("注文結果：")
print(response)

売るよ
注文結果：
<Response [200]>
